In [1]:
# dependencies
import pandas as pd
import requests
import json
from census import Census

# from uszipcode import SearchEngine

# US zipcode search (simple_zipcode=False provides polygon data)
#search = SearchEngine(simple_zipcode=True)

# census API key
from config import api_key

In [2]:
c = Census(api_key, year=2011)

In [3]:
white_pop_df = pd.DataFrame(c.acs5.state('B02001_002E', Census.ALL))

white_pop_df = white_pop_df.rename(columns={'B02001_002E': 'white_pop'})

white_pop_df = white_pop_df.set_index('state')

In [4]:
total_pop_df = pd.DataFrame(c.acs5.state('B01003_001E', Census.ALL))

total_pop_df = total_pop_df.rename(columns={'B01003_001E': 'total_pop'})

total_pop_df = total_pop_df.set_index('state')

total_pop_df

,total_pop
state,
01,4747424.0
02,700703.0
04,6337373.0
05,2895928.0
06,36969200.0
08,4966061.0
09,3558172.0
10,890856.0
11,593955.0


In [5]:
pop_df = total_pop_df.merge(white_pop_df, how='inner', left_index=True, right_index=True)

#pop_df = pd.merge(total_pop_df, white_pop_df, how='inner', on=total_pop_df['id']==white_pop_df['state'])

In [6]:
pop_df['pct_white'] = round((pop_df.white_pop / pop_df.total_pop *100), 2)

In [7]:
pop_df['fips'] = pop_df.index

In [8]:
min(pop_df.pct_white), max(pop_df.pct_white)

(24.95, 95.48)

In [17]:
pop_df = pop_df.astype({'fips': 'int32'})

In [18]:
pop_df.dtypes

total_pop    float64
white_pop    float64
pct_white    float64
fips           int32
dtype: object

In [19]:
pop_df.to_json('states2011.json', orient='records')